# Group 6 Project

In [1]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
dataset = load_dataset("mteb/twentynewsgroups-clustering", revision="6125ec4e24fa026cec8a478383ee943acfbd5449")
print(dataset)

/Users/thorkildkappel/Desktop/8. sem/data mining/project/Data_mining_G6/.venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    test: Dataset({
        features: ['sentences', 'labels'],
        num_rows: 10
    })
})


### Extracting variables:

In [2]:
test = dataset["test"]
texts_sep = list(test["sentences"])
labels_sep = list(test["labels"])

In [3]:
texts = []
for text in texts_sep:
    texts.extend(text)
print("number of texts: ", len(texts))

number of texts:  59545


### Preprocessing

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
tfidf = vectorizer.fit_transform(texts)

In [5]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2**6) #64
features = pca.fit_transform(tfidf)

In [6]:
print(features.shape)

(59545, 64)


In [101]:
test_features = features[0:10000]
print(test_features.shape)
print(test_features[9])

(10000, 64)
[-0.0122036   0.00465676 -0.02786715 -0.03424736 -0.01015916 -0.01203656
  0.00558481 -0.00539083  0.00787048 -0.00931066 -0.00828752 -0.00608116
  0.00453842  0.00353452  0.00530142  0.00531036 -0.01776302  0.01762594
  0.01250393  0.01038374 -0.02782703  0.00268476 -0.00948445 -0.03179292
  0.03242024 -0.02395956 -0.0251803   0.04113826 -0.00779184 -0.00238738
 -0.02991565 -0.02220932  0.06943595 -0.02690285  0.02067396 -0.01140224
  0.02607914 -0.02436134 -0.08032019  0.04227379 -0.06729909 -0.04073602
 -0.02197632  0.13467419  0.05814626 -0.08305549 -0.03099013 -0.07758375
 -0.00205484  0.17459134 -0.03919574  0.25248146 -0.09033909  0.0195749
 -0.03886773 -0.04697659 -0.02523195 -0.01196099  0.01696451  0.01980143
  0.06108541 -0.01048721  0.00636177  0.01668236]


In [103]:
from src.DBScan import DBScan

scanner =  DBScan(radius=0.1, min_dense=3)

clusters = scanner.make_clusters(test_features)

In [104]:
print(clusters[99])

62.0


### Using the DBScan elbow method